In [36]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import operator
from sklearn.linear_model import SGDClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from keras.callbacks import EarlyStopping

In [24]:
train_data = np.load("./resources/data_train.pkl", allow_pickle=True)
test_data = np.load("./resources/data_test.pkl", allow_pickle=True)
comment = train_data[0]
result = train_data[1]

In [25]:
lab = []
classes_name, classes_count = np.unique(result, return_counts=True)
for i in range(len(result)):
    lab.append(np.where(classes_name == result[i])[0][0])
lab = np.asarray(lab)

In [26]:
# TOKENIZER TWEET + NUMBERS
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
resultat_tweet_token = []
resultat_tweet = []
for i in range(len(comment)):
    tknzr = TweetTokenizer()
    resultat_tweet_token.append(tknzr.tokenize(comment[i].lower()))
    
    final = []
    for j in range(len(resultat_tweet_token[i])):
        if(resultat_tweet_token[i][j].isdigit()):
            if(len(resultat_tweet_token[i][j]) == 4):
                final.append('numerotypeDate')
            else:
                final.append('numerotype')
        elif(len(resultat_tweet_token[i][j]) == 1):
            if((resultat_tweet_token[i][j].isalpha() or resultat_tweet_token[i][j] == "$")):
                final.append(resultat_tweet_token[i][j])
        else:
            final.append(resultat_tweet_token[i][j])
    resultat_tweet_token[i] = final
                
                
    resultat_tweet.append(' '.join(resultat_tweet_token[i]))

In [27]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 500000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(resultat_tweet)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(resultat_tweet)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Found 75675 unique tokens.


In [29]:
Y = pd.get_dummies(lab)
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (70000, 20)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(
     X, Y, test_size=0.33, random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 100

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/home/jess/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42210 samples, validate on 4690 samples
Epoch 1/3
24500/42210 [================>.............] - ETA: 3:32 - loss: 2.8658 - accuracy: 0.0953

In [10]:
 def score_c(predictions, result):
        count = 0
        for i in range(len(predictions)):
            if(predictions[i] == result[i]):
                count += 1
        return 100*count/len(predictions)

In [11]:
m = model.predict(X_test)

In [12]:
prob = []
for i in range(len(m)):
    prob.append(np.argmax(m[i]))
    
res = []
for i in range(y_test.shape[0]):
    res.append(y_test.iloc[i,:].idxmax())

In [13]:
score_c(prob, res)

46.8733153638814